In [ ]:
library(tidyverse)
library(coda)
library(glue)

options(repr.plot.width = 15, repr.plot.height = 8)
theme_set(theme_gray(base_size = 20))


In [ ]:
samples <- read_csv("experiment_samples.csv") |>
    rename(Iteration = `...1`) |>
    pivot_longer(starts_with("birth_rate"),
        names_to = "Parameter",
        values_to = "Sample"
    ) |>
    mutate(tree = factor(tree))

truth <- tibble(
    Parameter = c("birth_rate1", "birth_rate2"),
    Truth = c(1, 4)
)


In [ ]:
ggplot(samples, aes(Iteration, Sample, color = tree)) +
    geom_line() +
    facet_wrap(vars(Parameter)) +
    ggtitle("Traceplots")


In [ ]:
samples |>
    group_by(Parameter, tree) |>
    summarise(
        n = n(),
        ESS = effectiveSize(Sample),
        quantile_025 = quantile(Sample, 0.025),
        median = median(Sample),
        quantile_975 = quantile(Sample, 0.975),
    ) |>
    left_join(truth, by = "Parameter") |>
    mutate(
        `Relative error` = abs((median - Truth) / Truth),
        Coverage = quantile_025 <= Truth & Truth <= quantile_975
    ) |>
    group_by(Parameter) |>
    summarise(
        n = sum(n),
        ESS = sum(ESS) |> round(),
        `Relative error` = mean(`Relative error`),
        Coverage = glue("{100*mean(Coverage)}%")
    )


In [ ]:
left_join(samples, truth, by = "Parameter") |>
    ggplot(aes(Sample)) +
    stat_function(
        aes(fill = "Prior"),
        fun = dlnorm,
        args = list(meanlog = 1.5, sdlog = 1),
        geom = "area",
        alpha = 0.8
    ) +
    geom_histogram(
        aes(y = after_stat(density), fill = "Posterior"),
        alpha = 0.8
    ) +
    geom_vline(aes(xintercept = Truth),
        color = "red",
        linewidth = 1.5,
        linetype = "dashed"
    ) +
    lims(x = c(0, 6)) +
    scale_fill_manual(
        name = "",
        values = c("Prior" = "grey", "Posterior" = "steelblue")
    ) +
    facet_wrap(vars(Parameter))
